In [1]:
import os
from tqdm.notebook import tqdm
import glob
import numpy as np
import pandas as pd
import random
from collections import defaultdict
from nltk.tokenize import sent_tokenize, word_tokenize

# Read files and annotations

In [2]:
#This class is provided by i2b2 challenge. I use its functions to get annotations and text

class RecordTrack2(object):
    """Record for Track 2 class."""

    def __init__(self, file_path):
        """Initialize."""
        self.path = os.path.abspath(file_path)
        self.basename = os.path.basename(self.path)
        self.annotations = self._get_annotations()
        # self.text = self._get_text()

    @property
    def tags(self):
        return self.annotations['tags']

    @property
    def relations(self):
        return self.annotations['relations']

    def _get_annotations(self):
        """Return a dictionary with all the annotations in the .ann file."""
        annotations = defaultdict(dict)
        with open(self.path) as annotation_file:
            lines = annotation_file.readlines()
            for line_num, line in enumerate(lines):
                if line.strip().startswith('T'):
                    try:
                        tag_id, tag_m, tag_text = line.strip().split('\t')
                    except ValueError:
                        print(self.path, line)
                    if len(tag_m.split(' ')) == 3:
                        tag_type, tag_start, tag_end = tag_m.split(' ')
                    elif len(tag_m.split(' ')) == 4:
                        tag_type, tag_start, _, tag_end = tag_m.split(' ')
                    elif len(tag_m.split(' ')) == 5:
                        tag_type, tag_start, _, _, tag_end = tag_m.split(' ')
                    else:
                        print(self.path)
                        print(line)
                    tag_start, tag_end = int(tag_start), int(tag_end)
                    annotations['tags'][tag_id] = (tag_id, tag_start ,tag_end, tag_type, tag_text)
            for line_num, line in enumerate(lines):
                if line.strip().startswith('R'):
                    rel_id, rel_m = line.strip().split('\t')
                    rel_type, rel_arg1, rel_arg2 = rel_m.split(' ')
                    rel_arg1 = rel_arg1.split(':')[1]
                    rel_arg2 = rel_arg2.split(':')[1]
                    arg1 = annotations['tags'][rel_arg1]
                    arg2 = annotations['tags'][rel_arg2]
                    annotations['relations'][rel_id] = (arg1,arg2, rel_type)
        return annotations

    def get_text(self):
        """Return the text in the corresponding txt file."""
        path = self.path.replace('.ann', '.txt')
        with open(path) as text_file:
            text = text_file.read()
        return text


In [3]:
#reading data and create a list of RecordTrack2 class for each file
folder1 = 'training_20180910/'
anno = []
files1 = set([os.path.basename(f) for f in glob.glob(
    os.path.join(folder1, '*.ann'))])
for file in files1:
    anno.append(RecordTrack2(os.path.join(folder1, file)))

# Generate entities within sentence

Find the offset of each entity within that sentence. 

The offset provided by i2b2 is aginst the whole file.

In this section, we will locate the entity, tokenise sentense, and remove extra spaces

The offset I generated in this section is word offset (created by len(text.split()), feel free to change to character offset if needed. 


In [4]:
#function to generate the first entity in a sentence
def first_entity(text, start, end, entity):
        text_before = sent_tokenize(text[:start]) #all the text befere the start of the entity
        sent_num = len(text_before)
        word_before = text_before[-1].split() #words before entity in the current sentence 
        word_start = len(word_before) #start position
        word_end = word_start+len(entity.split()) #end position
        sent_after = sent_tokenize(text[end:])[0] #the left of current sentence
        sent_piece = ' '.join(word_before + entity.split()) #current sentence until the end of first entity, extra spaces removed by .split()
        return word_start, word_end, sent_after, sent_piece

In [5]:
#function to generate non-first entity in a sentence, same logic sa first_entity function
def entity_within_sent(text, word_end_before, end_before, start, end, entity):
    word_between = text[end_before:start].split()
    word_start = word_end_before + len(word_between)
    word_end = word_start+len(entity.split())
    sent_after = sent_tokenize(text[end:])[0]
    #print(sent_after)
    sent_piece = ' '.join(word_between + entity.split())
    return word_start, word_end, sent_after, sent_piece

In [6]:
tags_list = []
sents_dict = {}
for i, an in enumerate(tqdm(anno)):
    text = an.get_text()
    tags = pd.DataFrame(an.tags.values()).sort_values(1).values
    #skip used to mark how many entities we have processed within the previous sentence
    skip = 0
    for j, (en_id, start, end, concept, entity) in enumerate(tags):
        #we skip the entities we have processed from last sentence, it will bring us to the first entity in current sentence
        if skip>0:
            skip -= 1
            continue
        #remove entities that across sentences
        if len(sent_tokenize(text[start:end]))>1:
            continue
        word_start, word_end, sent_after, sent_piece = first_entity(text, start, end, entity)
        #'-'.join([str(i),str(j)]) is an unique idenfitier for each sentence in the dataset, 
        #i identify the document and j identify the sentence within the document
        tags_list.append([i, en_id, word_start, word_end, concept, entity, '-'.join([str(i),str(j)])])
        while j+skip+1<len(tags) and end+len(sent_after)>= tags[j+skip+1][1]:
            anno_curr = tags[j+skip+1]
            word_end_before = word_end
            end_before = end
            sent_piece_before = sent_piece
            en_id = anno_curr[0]
            start = anno_curr[1]
            end = anno_curr[2]
            concept = anno_curr[3]
            entity = anno_curr[4]
            word_start, word_end, sent_after, sent_piece = entity_within_sent(text, word_end_before, end_before, start, end, entity)
            sent_piece = ' '. join([sent_piece_before, sent_piece])
            tags_list.append([i, en_id, word_start, word_end, concept, entity, '-'.join([str(i),str(j)])])
            skip += 1
        sents_dict['-'.join([str(i),str(j)])] = sent_piece

  0%|          | 0/303 [00:00<?, ?it/s]

In [7]:
tags_list

[[0, 'T1', 15, 16, 'Drug', 'Zosyn', '0-0'],
 [0, 'T5', 17, 18, 'Drug', 'Quinolones', '0-0'],
 [0, 'T6', 19, 20, 'Drug', 'Ceftriaxone', '0-0'],
 [0, 'T16', 21, 22, 'Drug', 'Flagyl', '0-0'],
 [0, 'T19', 23, 24, 'Drug', 'Keppra', '0-0'],
 [0, 'T20', 25, 26, 'Drug', 'Lisinopril', '0-0'],
 [0, 'T23', 27, 28, 'Drug', 'Naprosyn', '0-0'],
 [0, 'T24', 52, 53, 'Reason', 'HTN', '0-0'],
 [0, 'T25', 54, 55, 'Drug', 'ACEI', '0-0'],
 [0, 'T213', 14, 15, 'Drug', 'Lisinopril', '0-9'],
 [0, 'T214', 15, 17, 'Strength', '20 mg', '0-9'],
 [0, 'T271', 18, 19, 'Reason', 'anxiety', '0-11'],
 [0, 'T2', 20, 21, 'Reason', 'dyscomfort', '0-11'],
 [0, 'T3', 3, 5, 'Strength', '5 mg', '0-13'],
 [0, 'T4', 6, 7, 'Drug', 'midazolam', '0-13'],
 [0, 'T7', 2, 4, 'Strength', '125 mg', '0-15'],
 [0, 'T11', 5, 6, 'Route', 'IV', '0-15'],
 [0, 'T9', 6, 7, 'Drug', 'solumedrol', '0-15'],
 [0, 'T10', 8, 10, 'Strength', '20 mg', '0-15'],
 [0, 'T8', 11, 12, 'Route', 'IV', '0-15'],
 [0, 'T12', 12, 13, 'Drug', 'famotidine', '0-15'],


In [8]:
(sents_dict)

{'0-0': 'Admission Date: [**2177-5-27**] Discharge Date: [**2177-5-29**] Date of Birth: [**2129-10-5**] Sex: F Service: MEDICINE Allergies: Zosyn / Quinolones / Ceftriaxone / Flagyl / Keppra / Lisinopril / Naprosyn Attending:[**First Name3 (LF) 562**] Chief Complaint: swelling of lips Major Surgical or Invasive Procedure: none History of Present Illness: 47F with asthma/COPD, atopic dermatitis, HTN on ACEI',
 '0-9': 'She denies any wheezing, stridor, shortness of breath and states that has been on Lisinopril 20 mg',
 '0-11': 'ENT tried scoping her to assess for vocal cord swelling, but she did not tolerate procedure secondarely to anxiety and dyscomfort',
 '0-13': 'ER tried giving 5 mg of midazolam',
 '0-15': 'Pt received 125 mg of IV solumedrol , 20 mg of IV famotidine , 25 mg of IV diphenydramine and epipen x 2',
 '0-26': 'She is has a tough access and ER could only get a small IV in R forearm (that is why they gave epineprhine IM',
 '0-28': 'She had been on an ACEI',
 '0-29': 'There

In [30]:
import json
with open("sents_dict.txt", "w") as fp:
    json.dump(sents_dict, fp)  # encode dict into JSON

In [9]:
entity_df = pd.DataFrame(tags_list, columns=['text_id','entity_id','start','end','concept','entity','sent_num']).set_index('entity_id')

In [10]:
entity_df['concept'].unique()

array(['Drug', 'Reason', 'Strength', 'Route', 'Dosage', 'ADE',
       'Frequency', 'Form', 'Duration'], dtype=object)

In [31]:
entity_df.to_csv('entity.csv', sep=',', encoding='utf-8')

In [19]:
'''
This is to generate relations to a specific format for one of my model, a relation triplet is formated as 
"start_position_1st_entity, end_position_1st_entity, start_position_2nd_entity, end_position_2nd_entity,
relation_type, 1st_entity_type, 2nd_entity_type" Each relation is concatenated by all the relation triplets
within the sentence, seperated by '|'
'''

relations = []
sents = []
for i, an in enumerate(tqdm(anno)):
    text = an.get_text()
    relation = ''
    entity_curr = entity_df[entity_df['text_id']==i]#['sent_num']
    curr_sent = 0
    relation_df = pd.DataFrame.from_dict(an.relations,orient='index')#.values()
    relation_df['head_entity'] = relation_df[1].apply(lambda x: int(x[1]))
    relation_df = relation_df.sort_values(['head_entity']).drop('head_entity',axis = 1)
    for k in relation_df.values:
        if k[0][0] not in entity_curr.index or k[1][0] not in entity_curr.index:
            continue
        e1, e2, rel = entity_curr.loc[k[0][0]], entity_curr.loc[k[1][0]], k[2]

 

        if e1['sent_num'] != e2['sent_num']:
            continue
        if curr_sent == 0:
            relation = ' '.join([str(e1['start']), str(e1['end']), str(e2['start']), str(e2['end']),
                                   '/'+rel+'/'+k[0][3]+'/'+k[1][3], '|'])
            curr_sent = e1['sent_num']
        elif e1['sent_num'] == curr_sent:           
            relation = ' '.join([relation, str(e1['start']), str(e1['end']), str(e2['start']), str(e2['end']),
                                   '/'+rel+'/'+k[0][3]+'/'+k[1][3], '|'])
        else:
            #print(curr_sent)
            relations.append((curr_sent,relation[:-2]))  
            sents.append((curr_sent,sents_dict[curr_sent]))
            curr_sent = e1['sent_num']
            relation = ' '.join([str(e1['start']), str(e1['end']), str(e2['start']), str(e2['end']),
                                   '/'+rel+'/'+k[0][3]+'/'+k[1][3],'|'])
    if curr_sent == 0:
        continue
    relations.append((curr_sent,relation[:-2]))  
    sents.append((curr_sent,sents_dict[curr_sent]))
    relation = ' '.join([str(e1['start']), str(e1['end']), str(e2['start']), str(e2['end']),
                           '/'+rel+'/'+k[0][3]+'/'+k[1][3],'|'])



  0%|          | 0/303 [00:00<?, ?it/s]

In [20]:
relations[0:7]

[('0-0', '52 53 54 55 /Reason-Drug/Reason/Drug'),
 ('0-9', '15 17 14 15 /Strength-Drug/Strength/Drug'),
 ('0-13', '3 5 6 7 /Strength-Drug/Strength/Drug'),
 ('0-15',
  '2 4 6 7 /Strength-Drug/Strength/Drug | 5 6 6 7 /Route-Drug/Route/Drug | 11 12 12 13 /Route-Drug/Route/Drug | 8 10 12 13 /Strength-Drug/Strength/Drug | 14 16 18 19 /Strength-Drug/Strength/Drug | 17 18 18 19 /Route-Drug/Route/Drug | 22 23 20 21 /Dosage-Drug/Dosage/Drug'),
 ('0-26', '23 24 22 23 /Route-Drug/Route/Drug'),
 ('0-30', '10 11 8 9 /ADE-Drug/ADE/Drug'),
 ('0-40',
  '1 2 5 6 /Reason-Drug/Reason/Drug | 1 2 7 8 /Reason-Drug/Reason/Drug')]

In [24]:
import re
print(re.findall('\d+', relations[6][1]))

['1', '2', '5', '6', '1', '2', '7', '8']


In [21]:
sents[0:7]

[('0-0',
  'Admission Date: [**2177-5-27**] Discharge Date: [**2177-5-29**] Date of Birth: [**2129-10-5**] Sex: F Service: MEDICINE Allergies: Zosyn / Quinolones / Ceftriaxone / Flagyl / Keppra / Lisinopril / Naprosyn Attending:[**First Name3 (LF) 562**] Chief Complaint: swelling of lips Major Surgical or Invasive Procedure: none History of Present Illness: 47F with asthma/COPD, atopic dermatitis, HTN on ACEI'),
 ('0-9',
  'She denies any wheezing, stridor, shortness of breath and states that has been on Lisinopril 20 mg'),
 ('0-13', 'ER tried giving 5 mg of midazolam'),
 ('0-15',
  'Pt received 125 mg of IV solumedrol , 20 mg of IV famotidine , 25 mg of IV diphenydramine and epipen x 2'),
 ('0-26',
  'She is has a tough access and ER could only get a small IV in R forearm (that is why they gave epineprhine IM'),
 ('0-30',
  'She was been seen by allergy for likely naproxen induced angioedema'),
 ('0-40', '#. HTN - Pt on ACEI and beta-blocker')]

In [23]:
sents[6][1].split()

['#.', 'HTN', '-', 'Pt', 'on', 'ACEI', 'and', 'beta-blocker']

In [25]:
i_relation = relations[6][1].split("| ")
sample = []
for i in i_relation:
    relation_sample = []
    relation_sample.append((int(i.split(" ")[0]),int(i.split(" ")[1])))
    relation_sample.append((int(i.split(" ")[2]),int(i.split(" ")[3])))
    relation_sample.append(i.split(" ")[4].split("/")[1])
    sample.append(relation_sample)
print(sample)

[[(1, 2), (5, 6), 'Reason-Drug'], [(1, 2), (7, 8), 'Reason-Drug']]


In [26]:
relation_data = []
for i in range(len(sents)):
    sample = []
    sample.append(sents[i][0])
    sample.append(sents[i][1].split())
    i_relation = relations[i][1].split("| ")
    i_sample = []
    for i in i_relation:
        relation_sample = []
        relation_sample.append((int(i.split(" ")[0]),int(i.split(" ")[1])))
        relation_sample.append((int(i.split(" ")[2]),int(i.split(" ")[3])))
        relation_sample.append(i.split(" ")[4].split("/")[1])
        i_sample.append(relation_sample)
    sample.append(i_sample)
    relation_data.append(sample)

In [29]:
relation_data[0]

['0-0',
 ['Admission',
  'Date:',
  '[**2177-5-27**]',
  'Discharge',
  'Date:',
  '[**2177-5-29**]',
  'Date',
  'of',
  'Birth:',
  '[**2129-10-5**]',
  'Sex:',
  'F',
  'Service:',
  'MEDICINE',
  'Allergies:',
  'Zosyn',
  '/',
  'Quinolones',
  '/',
  'Ceftriaxone',
  '/',
  'Flagyl',
  '/',
  'Keppra',
  '/',
  'Lisinopril',
  '/',
  'Naprosyn',
  'Attending:[**First',
  'Name3',
  '(LF)',
  '562**]',
  'Chief',
  'Complaint:',
  'swelling',
  'of',
  'lips',
  'Major',
  'Surgical',
  'or',
  'Invasive',
  'Procedure:',
  'none',
  'History',
  'of',
  'Present',
  'Illness:',
  '47F',
  'with',
  'asthma/COPD,',
  'atopic',
  'dermatitis,',
  'HTN',
  'on',
  'ACEI'],
 [[(52, 53), (54, 55), 'Reason-Drug']]]

In [28]:
import json
with open("relation.txt", "w") as fp:
    json.dump(relation_data, fp)